In [ ]:
import gradio as gr
import networkx as nx
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import json
import io
import base64
import tempfile
import os
from collections import deque
import heapq

# ==================== GRAPH OPERATIONS ====================
class GraphOperations:
    def __init__(self):
        self.graph = None
        self.directed = False
    
    def set_graph(self, G, directed=False):
        self.graph = G
        self.directed = directed
    
    def shortest_path(self, start, end):
        if not self.graph.has_node(start) or not self.graph.has_node(end):
            return None, float('inf')
        try:
            path = nx.dijkstra_path(self.graph, start, end)
            length = nx.dijkstra_path_length(self.graph, start, end)
            return path, length
        except nx.NetworkXNoPath:
            return None, float('inf')
    
    def bfs_traversal(self, start):
        visited = []
        queue = deque([start])
        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.append(node)
                for neighbor in self.graph.neighbors(node):
                    if neighbor not in visited and neighbor not in queue:
                        queue.append(neighbor)
        return visited
    
    def dfs_traversal(self, start):
        visited = []
        stack = [start]
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.append(node)
                for neighbor in reversed(list(self.graph.neighbors(node))):
                    if neighbor not in visited:
                        stack.append(neighbor)
        return visited
    
    def is_bipartite(self):
        try:
            return nx.is_bipartite(self.graph)
        except:
            return False
    
    def to_adjacency_matrix(self):
        nodes = sorted(self.graph.nodes())
        n = len(nodes)
        matrix = [[0] * n for _ in range(n)]
        node_index = {node: i for i, node in enumerate(nodes)}
        for u, v, data in self.graph.edges(data=True):
            weight = data.get('weight', 1)
            i, j = node_index[u], node_index[v]
            matrix[i][j] = weight
            if not self.directed:
                matrix[j][i] = weight
        return matrix, nodes
    
    def to_adjacency_list(self):
        adj_list = {}
        for node in self.graph.nodes():
            neighbors = []
            for neighbor in self.graph.neighbors(node):
                weight = self.graph[node][neighbor].get('weight', 1)
                neighbors.append((neighbor, weight))
            adj_list[node] = neighbors
        return adj_list
    
    def to_edge_list(self):
        edges = []
        for u, v, data in self.graph.edges(data=True):
            weight = data.get('weight', 1)
            edges.append((u, v, weight))
        return edges
    
    def prim_mst(self):
        if not nx.is_connected(self.graph.to_undirected()):
            return None
        mst_edges = []
        visited = set()
        start_node = list(self.graph.nodes())[0]
        visited.add(start_node)
        edges = []
        for neighbor in self.graph.neighbors(start_node):
            weight = self.graph[start_node][neighbor].get('weight', 1)
            heapq.heappush(edges, (weight, start_node, neighbor))
        while edges and len(visited) < len(self.graph.nodes()):
            weight, u, v = heapq.heappop(edges)
            if v not in visited:
                visited.add(v)
                mst_edges.append((u, v, weight))
                for neighbor in self.graph.neighbors(v):
                    if neighbor not in visited:
                        w = self.graph[v][neighbor].get('weight', 1)
                        heapq.heappush(edges, (w, v, neighbor))
        return mst_edges
    
    def kruskal_mst(self):
        parent = {}
        def find(node):
            if parent[node] != node:
                parent[node] = find(parent[node])
            return parent[node]
        def union(u, v):
            root_u = find(u)
            root_v = find(v)
            if root_u != root_v:
                parent[root_v] = root_u
                return True
            return False
        for node in self.graph.nodes():
            parent[node] = node
        edges = []
        for u, v, data in self.graph.edges(data=True):
            weight = data.get('weight', 1)
            edges.append((weight, u, v))
        edges.sort()
        mst_edges = []
        for weight, u, v in edges:
            if find(u) != find(v):
                union(u, v)
                mst_edges.append((u, v, weight))
        return mst_edges
    
    def ford_fulkerson(self, source, sink):
        R = nx.DiGraph() if self.directed else nx.Graph()
        for u, v, data in self.graph.edges(data=True):
            capacity = data.get('weight', 1)
            R.add_edge(u, v, capacity=capacity, flow=0)
            if not self.directed:
                R.add_edge(v, u, capacity=capacity, flow=0)
        max_flow = 0
        while True:
            visited = {source: None}
            queue = deque([source])
            found = False
            while queue and not found:
                u = queue.popleft()
                for v in R.neighbors(u):
                    if v not in visited and R[u][v]['capacity'] - R[u][v]['flow'] > 0:
                        visited[v] = u
                        if v == sink:
                            found = True
                            break
                        queue.append(v)
            if not found:
                break
            path_flow = float('inf')
            v = sink
            while v != source:
                u = visited[v]
                path_flow = min(path_flow, R[u][v]['capacity'] - R[u][v]['flow'])
                v = u
            v = sink
            while v != source:
                u = visited[v]
                R[u][v]['flow'] += path_flow
                R[v][u]['flow'] -= path_flow
                v = u
            max_flow += path_flow
        return max_flow

# ==================== UTILS ====================
def draw_graph(G, highlight_nodes=None, highlight_edges=None, title="", directed=False):
    plt.figure(figsize=(8, 6))
    pos = nx.spring_layout(G, seed=42)
    
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500, alpha=0.8)
    nx.draw_networkx_labels(G, pos)
    
    edge_colors = ['gray'] * len(G.edges())
    edge_widths = [1] * len(G.edges())
    
    if highlight_edges:
        edge_list = list(G.edges())
        for i, edge in enumerate(edge_list):
            if edge in highlight_edges or (edge[1], edge[0]) in highlight_edges:
                edge_colors[i] = 'red'
                edge_widths[i] = 3
    
    if directed:
        nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=edge_widths,
                              arrows=True, arrowstyle='-|>', arrowsize=20,
                              connectionstyle='arc3,rad=0.0')
    else:
        nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=edge_widths,
                              connectionstyle='arc3,rad=0.0')
    
    if highlight_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=highlight_nodes, 
                              node_color='red', node_size=700)
    
    edge_labels = nx.get_edge_attributes(G, 'weight')
    if edge_labels:
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    
    plt.title(title)
    plt.axis('off')
    
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', dpi=100)
    plt.close()
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode()
    return f"data:image/png;base64,{img_str}"

# ==================== GLOBAL STATE ====================
current_graph = nx.Graph()
is_directed = False
graph_ops = GraphOperations()

# ==================== HANDLERS ====================
def safe_int_convert(val):
    try:
        return int(float(val))
    except:
        return 0

def create_graph_handler(text, directed):
    global current_graph, is_directed, graph_ops
    is_directed = directed
    edges = []
    
    for line in text.strip().split('\n'):
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        if len(parts) >= 2:
            try:
                u = safe_int_convert(parts[0])
                v = safe_int_convert(parts[1])
                w = float(parts[2]) if len(parts) > 2 else 1.0
                edges.append((u, v, w))
            except:
                continue
    
    if not edges:
        return "❌ Không có dữ liệu hợp lệ", None
    
    current_graph = nx.DiGraph() if directed else nx.Graph()
    for u, v, w in edges:
        current_graph.add_edge(u, v, weight=w)
    
    graph_ops.set_graph(current_graph, directed)
    img = draw_graph(current_graph, title=f"Đã tạo {len(edges)} cạnh", directed=directed)
    return f"✅ Tạo thành công {len(edges)} cạnh", img

def shortest_path_handler(start, end):
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    try:
        start = int(start)
        end = int(end)
        path, length = graph_ops.shortest_path(start, end)
        if path:
            path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
            img = draw_graph(current_graph, highlight_nodes=path, 
                           highlight_edges=path_edges, directed=is_directed,
                           title=f"Đường đi: {path} (dài: {length})")
            return f"📏 Đường đi: {' → '.join(map(str, path))}\n📊 Độ dài: {length}", img
        else:
            img = draw_graph(current_graph, directed=is_directed)
            return "⚠ Không tìm thấy đường đi", img
    except:
        img = draw_graph(current_graph, directed=is_directed)
        return "❌ Node không hợp lệ", img

def bfs_handler(start):
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    try:
        start = int(start)
        bfs_nodes = graph_ops.bfs_traversal(start)
        bfs_tree = nx.bfs_tree(current_graph, start)
        bfs_edges = list(bfs_tree.edges())
        img = draw_graph(current_graph, highlight_nodes=bfs_nodes,
                        highlight_edges=bfs_edges, directed=is_directed,
                        title=f"BFS từ node {start}")
        return f"🔄 BFS: {bfs_nodes}", img
    except:
        img = draw_graph(current_graph, directed=is_directed)
        return "❌ Node không hợp lệ", img

def dfs_handler(start):
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    try:
        start = int(start)
        dfs_nodes = graph_ops.dfs_traversal(start)
        dfs_tree = nx.dfs_tree(current_graph, start)
        dfs_edges = list(dfs_tree.edges())
        img = draw_graph(current_graph, highlight_nodes=dfs_nodes,
                        highlight_edges=dfs_edges, directed=is_directed,
                        title=f"DFS từ node {start}")
        return f"🔍 DFS: {dfs_nodes}", img
    except:
        img = draw_graph(current_graph, directed=is_directed)
        return "❌ Node không hợp lệ", img

def bipartite_handler():
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    try:
        is_bip = graph_ops.is_bipartite()
        result = "✅ Là đồ thị 2 phía" if is_bip else "❌ Không phải đồ thị 2 phía"
        img = draw_graph(current_graph, directed=is_directed, title=result)
        return result, img
    except:
        img = draw_graph(current_graph, directed=is_directed)
        return "⚠ Không thể kiểm tra", img

def prim_handler():
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    try:
        mst_edges = graph_ops.prim_mst()
        if mst_edges:
            total_weight = sum(weight for _, _, weight in mst_edges)
            mst_edge_list = [(u, v) for u, v, _ in mst_edges]
            img = draw_graph(current_graph, highlight_edges=mst_edge_list,
                           directed=is_directed,
                           title=f"Prim MST - Tổng: {total_weight}")
            result = f"✅ Prim MST:\n" + "\n".join([f"  {u}→{v} (w={w})" for u, v, w in mst_edges])
            result += f"\n📊 Tổng: {total_weight}"
            return result, img
        return "⚠ Đồ thị không liên thông", None
    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def kruskal_handler():
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    try:
        mst_edges = graph_ops.kruskal_mst()
        if mst_edges:
            total_weight = sum(weight for _, _, weight in mst_edges)
            mst_edge_list = [(u, v) for u, v, _ in mst_edges]
            img = draw_graph(current_graph, highlight_edges=mst_edge_list,
                           directed=is_directed,
                           title=f"Kruskal MST - Tổng: {total_weight}")
            result = f"✅ Kruskal MST:\n" + "\n".join([f"  {u}→{v} (w={w})" for u, v, w in mst_edges])
            result += f"\n📊 Tổng: {total_weight}"
            return result, img
        return "⚠ Đồ thị không liên thông", None
    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def ford_fulkerson_handler(source, sink):
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    try:
        source = int(source)
        sink = int(sink)
        max_flow = graph_ops.ford_fulkerson(source, sink)
        img = draw_graph(current_graph, directed=is_directed,
                        title=f"Ford-Fulkerson: {source}→{sink} = {max_flow}")
        return f"🌊 Luồng cực đại {source}→{sink}: {max_flow}", img
    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def fleury_handler(start_node):
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị", None
    try:
        start = int(start_node)
        if nx.is_eulerian(current_graph):
            euler_circuit = list(nx.eulerian_circuit(current_graph, source=start))
            path = [start]
            for u, v in euler_circuit:
                if v not in path:
                    path.append(v)
            path_edges = [(u, v) for u, v in euler_circuit]
            img = draw_graph(current_graph, highlight_nodes=path,
                           highlight_edges=path_edges, directed=is_directed,
                           title=f"Fleury - Chu trình Euler")
            result = f"✅ Chu trình Euler:\n" + "\n".join([f"  {u}→{v}" for u, v in euler_circuit])
            return result, img
        return "⚠ Không có chu trình Euler", None
    except Exception as e:
        return f"❌ Lỗi: {str(e)}", None

def hierholzer_handler(start_node):
    return fleury_handler(start_node)  # Tương tự

def convert_handler(format_type):
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị"
    try:
        if format_type == "Ma trận kề":
            matrix, nodes = graph_ops.to_adjacency_matrix()
            result = "Ma trận kề:\n   " + " ".join(str(n) for n in nodes) + "\n"
            for i, row in enumerate(matrix):
                result += f"{nodes[i]}: " + " ".join(str(x) for x in row) + "\n"
        elif format_type == "Danh sách kề":
            adj_list = graph_ops.to_adjacency_list()
            result = "Danh sách kề:\n"
            for node in sorted(adj_list.keys()):
                neighbors = adj_list[node]
                neighbor_str = ", ".join(f"{n}({w})" for n, w in neighbors)
                result += f"{node}: {neighbor_str}\n"
        else:
            edges = graph_ops.to_edge_list()
            result = "Danh sách cạnh:\n"
            for u, v, w in edges:
                result += f"({u}, {v}, {w})\n"
        return result
    except Exception as e:
        return f"❌ Lỗi: {str(e)}"

def save_handler():
    if not current_graph.nodes():
        return "❌ Chưa có đồ thị"
    edges = [(u, v, current_graph[u][v].get('weight', 1)) for u, v in current_graph.edges()]
    data = {"directed": is_directed, "nodes": list(current_graph.nodes()), "edges": edges}
    return json.dumps(data, indent=2)

def load_handler(json_str):
    try:
        data = json.loads(json_str)
        global current_graph, is_directed, graph_ops
        is_directed = data.get("directed", False)
        current_graph = nx.DiGraph() if is_directed else nx.Graph()
        for u, v, w in data.get("edges", []):
            current_graph.add_edge(u, v, weight=w)
        graph_ops.set_graph(current_graph, is_directed)
        img = draw_graph(current_graph, directed=is_directed, title="Đồ thị đã tải")
        return "✅ Đã tải thành công", img
    except:
        return "❌ JSON không hợp lệ", None

# ==================== GRADIO UI ====================
with gr.Blocks(title="Graph Visualizer", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📊 **TRÌNH XỬ LÝ ĐỒ THỊ**")
    gr.Markdown("Nhập đồ thị và thực hiện các thuật toán cơ bản & nâng cao")
    
    with gr.Tabs():
        with gr.Tab("📝 Nhập đồ thị"):
            with gr.Row():
                with gr.Column(scale=1):
                    input_text = gr.Textbox(
                        label="Mỗi dòng: u v [weight]",
                        placeholder="0 1 5\n0 2 3\n1 2 2",
                        lines=10,
                        value="0 1 5\n0 2 3\n1 2 2"
                    )
                    with gr.Row():
                        directed_cb = gr.Checkbox(label="Đồ thị có hướng", value=False)
                        create_btn = gr.Button("Tạo đồ thị", variant="primary", size="lg")
                    status = gr.Textbox(label="Trạng thái", interactive=False)
                with gr.Column(scale=1):
                    output_img = gr.Image(label="Đồ thị")
            create_btn.click(create_graph_handler, [input_text, directed_cb], [status, output_img])
        
        with gr.Tab("🔍 Thuật toán cơ bản"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### **Đường đi ngắn nhất**")
                    with gr.Row():
                        start_node = gr.Number(label="Node bắt đầu", value=0, precision=0)
                        end_node = gr.Number(label="Node kết thúc", value=1, precision=0)
                    dijkstra_btn = gr.Button("Tìm đường đi", variant="primary")
                    dijkstra_result = gr.Textbox(label="Kết quả")
                    
                    gr.Markdown("### **Duyệt đồ thị**")
                    traversal_start = gr.Number(label="Node bắt đầu", value=0, precision=0)
                    with gr.Row():
                        bfs_btn = gr.Button("BFS")
                        dfs_btn = gr.Button("DFS")
                    traversal_result = gr.Textbox(label="Kết quả duyệt")
                    
                    gr.Markdown("### **Kiểm tra tính chất**")
                    bipartite_btn = gr.Button("Kiểm tra 2 phía")
                    bipartite_result = gr.Textbox(label="Kết quả")
                with gr.Column():
                    algo_img = gr.Image(label="Kết quả trực quan")
            
            dijkstra_btn.click(shortest_path_handler, [start_node, end_node], [dijkstra_result, algo_img])
            bfs_btn.click(bfs_handler, [traversal_start], [traversal_result, algo_img])
            dfs_btn.click(dfs_handler, [traversal_start], [traversal_result, algo_img])
            bipartite_btn.click(bipartite_handler, [], [bipartite_result, algo_img])
        
        with gr.Tab("🚀 Thuật toán nâng cao"):
            with gr.Row():
                with gr.Column(scale=1):
                    algo_choice = gr.Radio(
                        choices=["Prim", "Kruskal", "Ford-Fulkerson", "Fleury", "Hierholzer"],
                        label="Chọn thuật toán",
                        value="Prim"
                    )
                    with gr.Group() as param_group:
                        source_input = gr.Number(label="Source", value=0, precision=0, visible=False)
                        sink_input = gr.Number(label="Sink", value=1, precision=0, visible=False)
                        start_input = gr.Number(label="Node bắt đầu", value=0, precision=0, visible=False)
                    
                    def update_inputs(algo):
                        vis_source = (algo == "Ford-Fulkerson")
                        vis_sink = (algo == "Ford-Fulkerson")
                        vis_start = (algo in ["Fleury", "Hierholzer"])
                        return [gr.update(visible=vis_source), gr.update(visible=vis_sink), gr.update(visible=vis_start)]
                    
                    algo_choice.change(update_inputs, [algo_choice], [source_input, sink_input, start_input])
                    run_algo_btn = gr.Button("Chạy thuật toán", variant="primary", size="lg")
                with gr.Column(scale=1):
                    advanced_result = gr.Textbox(label="Kết quả thuật toán", lines=6)
                    advanced_img = gr.Image(label="Trực quan hóa")
            
            def run_advanced_algo(algo, source, sink, start):
                if algo == "Prim": return prim_handler()
                elif algo == "Kruskal": return kruskal_handler()
                elif algo == "Ford-Fulkerson": 
                    if source is None or sink is None: return "⚠ Vui lòng nhập source và sink", None
                    return ford_fulkerson_handler(source, sink)
                elif algo == "Fleury": 
                    if start is None: return "⚠ Vui lòng nhập node bắt đầu", None
                    return fleury_handler(start)
                elif algo == "Hierholzer": 
                    if start is None: return "⚠ Vui lòng nhập node bắt đầu", None
                    return hierholzer_handler(start)
                return "⚠ Vui lòng chọn thuật toán", None
            
            run_algo_btn.click(run_advanced_algo, [algo_choice, source_input, sink_input, start_input], [advanced_result, advanced_img])
        
        with gr.Tab("🔄 Chuyển đổi"):
            format_type = gr.Radio(
                choices=["Ma trận kề", "Danh sách kề", "Danh sách cạnh"],
                label="Chọn định dạng",
                value="Danh sách cạnh"
            )
            convert_btn = gr.Button("Chuyển đổi", variant="primary")
            conversion_output = gr.Textbox(label="Kết quả", lines=10)
            convert_btn.click(convert_handler, [format_type], [conversion_output])
        
        with gr.Tab("💾 Lưu/Tải"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### **Lưu đồ thị**")
                    save_btn = gr.Button("Xuất JSON", variant="primary")
                    json_output = gr.Textbox(label="Dữ liệu JSON", lines=8)
                    save_btn.click(save_handler, [], [json_output])
                with gr.Column():
                    gr.Markdown("### **Tải đồ thị**")
                    json_input = gr.Textbox(
                        label="Dán JSON ở đây",
                        placeholder='{"directed": false, "edges": [[0,1,5], [0,2,3]]}',
                        lines=8
                    )
                    load_btn = gr.Button("Tải từ JSON")
                    load_status = gr.Textbox(label="Trạng thái")
                    load_btn.click(load_handler, [json_input], [load_status, output_img])
    
    gr.Markdown("---")
    gr.Markdown("""
    ### 📌 **Hướng dẫn nhanh:**
    1. **Tab 1**: Nhập đồ thị (mỗi dòng: `u v weight`)
    2. **Tab 2**: Thuật toán cơ bản (Dijkstra, BFS, DFS, 2 phía)
    3. **Tab 3**: Thuật toán nâng cao (Prim, Kruskal, Ford-Fulkerson, Fleury, Hierholzer)
    4. **Tab 4**: Chuyển đổi định dạng
    5. **Tab 5**: Lưu/tải đồ thị
    """)

if __name__ == "__main__":
    demo.launch()

ImportError: cannot import name 'draw_graph' from 'utils' (e:\Desktop\Các môn học năm 2\Cau truc roi rac\graph_app\utils.py)